In [17]:
import sys
import copy as cp
import tensorflow as tf
import numpy as np
import importlib

import parameters
import helper_functions
import model as tem

In [18]:
# Create Directories
gen_path, train_path, model_path, save_path, script_path = helper_functions.make_directories()

# Initialise Parameters
pars = parameters.default_params()
pars_orig = pars.copy()

tf.reset_default_graph()
with tf.name_scope('Inputs'):
    it_num = tf.placeholder(tf.float32, shape=(), name='it_num')
    seq_ind = tf.placeholder(tf.float32, shape=(), name='seq_ind')
    rnn = tf.placeholder(tf.float32, shape=(pars['batch_size'], pars['p_size'], pars['p_size']), name='rnn')
    rnn_inv = tf.placeholder(tf.float32, shape=(pars['batch_size'], pars['p_size'], pars['p_size']), name='rnn')
    x1_two_hot = tf.placeholder(tf.float32, shape=(pars['batch_size'], pars['s_size_comp'], pars['seq_len']),
                                name='x1_two_hot')
    x1 = tf.placeholder(tf.float32, shape=(pars['batch_size'], pars['s_size'], pars['seq_len']), name='x')
    g_ = tf.placeholder(tf.float32, shape=(pars['batch_size'], pars['g_size']), name='g_')
    x_ = tf.placeholder(tf.float32, shape=(pars['batch_size'], pars['s_size_comp'] * pars['n_freq']), name='x_')
    sh = tf.placeholder(tf.float32, shape=(pars['batch_size'], pars['s_size']), name='shiny')
    # need to feed in lists etc
    d0 = tf.placeholder(tf.float32, shape=(pars['batch_size'], pars['n_actions'], pars['seq_len']), name='d')
    s_visi = tf.placeholder(tf.float32, shape=(pars['batch_size'], pars['seq_len']), name='s_visited')
    no_d = tf.placeholder(tf.float32, shape=(pars['batch_size']), name='no_direc_batch')
    x_two_hot = tf.unstack(x1_two_hot, axis=2)
    x = tf.unstack(x1, axis=2)
    d = tf.unstack(d0, axis=2)
    s_vis = tf.unstack(s_visi, axis=1)

# Initialise Model
model = tem.TEM(x, x_, x_two_hot, g_, d, rnn, rnn_inv, it_num, seq_ind, s_vis, sh, no_d, pars)

# Initialise Model Variables
fetches_all, fetches_summary, fetches_all_, fetches_summary_ = [], [], [], []
fetches_all.extend([model.g, model.p, model.p_g, model.x_gt, model.x_, model.A, model.A_inv, model.accuracy_gt,
                    model.train_op_all])
fetches_all_.extend([model.g, model.p, model.p_g, model.x_gt, model.x_, model.A, model.A_inv, model.accuracy_gt,
                     model.temp])
fetches_summary.extend([model.lx_p, model.lx_g, model.lx_gt, model.lp, model.lg, model.g, model.p, model.p_g,
                        model.x_gt, model.x_, model.A, model.A_inv, model.accuracy_p, model.accuracy_g,
                        model.accuracy_gt, model.merged, model.train_op_all])
fetches_summary_.extend([model.lx_p, model.lx_g, model.lx_gt, model.lp, model.lg, model.g, model.p, model.p_g,
                         model.x_gt, model.x_, model.A, model.A_inv, model.accuracy_p, model.accuracy_g,
                         model.accuracy_gt, model.merged, model.temp])

AttributeError: 'TEM' object has no attribute 'train_op_all'

In [1]:
# CREATE SESSION
sess = tf.InteractiveSession()
saver = tf.train.Saver(max_to_keep=1)  # saves variables learned during training
train_writer = tf.summary.FileWriter(train_path, sess.graph)
tf.global_variables_initializer().run()
tf.get_default_graph().finalize()

# Initialise Variables
lx_ps, lx_gs, lx_gts, lps, lgs = [], [], [], [], []
accs_p, accs_g, accs_gt = [], [], []
check_link_inference = False
acc_p, acc_g, acc_gt, seq_index, rn = 0, 0, 0, 0, 0
correct_link, positions_link, positions, visited, state_guess = None, None, None, None, None
position_all, direc_all = None, None
gs_all, ps_all, ps_gen_all, xs_all, gs_timeseries, ps_timeseries, pos_timeseries = \
    [None] * pars['n_envs_save'], [None] * pars['n_envs_save'], [None] * pars['n_envs_save'], \
    [None] * pars['n_envs_save'], [None] * pars['n_envs_save'], [None] * pars['n_envs_save'], \
    [None] * pars['n_envs_save']
cell_timeseries, prev_cell_timeseries = None, None
accs_x_to, accs_x_from = [None] * pars['n_envs_save'], [None] * pars['n_envs_save']
save_needed, save_ticker, summary_needed, summary_ticker, save_model = False, False, False, False, False
table, _ = helper_functions.combins_table(pars['s_size_comp'], 2)
n_restart = pars['restart_max'] + pars['curriculum_steps']
T, F, L, P = None, None, None, None
adjs, trans, states_mat, shiny_s, shiny_states = None, None, None, None, None
a_rnn, a_rnn_inv = None, None
gs, ps, x_s, x_data, start_state, prev_direc = None, None, None, None, None, None
n_walk = 100
index = 0

for i in range(pars['train_iters']):
    print('Iter {} of {}'.format(i, n_walk))
    # Curriculum of Behaviour Types
    pars, shiny_s, rn, n_restart, no_direc_batch = helper_functions.curriculum(pars_orig, pars, n_restart)

    # Make Environment
    adjs, trans, states_mat, shiny_states = helper_functions.make_environments(pars)

    # Create Hebbian Matrices
    a_rnn, a_rnn_inv = helper_functions.initialise_hebb(pars)

    # Initialise all other Variables
    gs, x_s, x_data, start_state, prev_direc, visited = helper_functions.initialise_variables(pars, adjs)

    # Collect Walking Data
    position_all, direc_all = helper_functions.get_walking_data(start_state, adjs, trans, prev_direc, shiny_states, n_walk * pars['n_walk'], pars)

    # Run Model
    p, g, x_, x_p, x_g, x_gt = tem.TEM.obs_to_state()



NameError: name 'tf' is not defined